<a href="https://colab.research.google.com/github/sagawritescode/ENDTwoPointOPhase1/blob/main/Sagar_Sentiment_Analysis_using_LSTM_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [2]:
import random
import torch, torchtext.legacy
from torchtext.legacy import data

PyTreebank

In [4]:
!pip install pytreebank

  Created wheel for pytreebank: filename=pytreebank-0.2.7-cp37-none-any.whl size=37070 sha256=6f7f1cef95bc243d565ebc1a848264a4b7596c5c7875a1d32f32653a9bc19c73
  Stored in directory: /root/.cache/pip/wheels/e0/b6/91/e9edcdbf464f623628d5c3aa9de28888c726e270b9a29f2368
Successfully built pytreebank


In [5]:
import pytreebank
dataset = pytreebank.load_sst()

In [6]:
Review = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [7]:
fields = [('reviews', Review),('labels',Label)]

In [8]:
# Use list to collect data as it is more efficient for appending
totaldata = []

for databatch in [dataset["train"], dataset["dev"], dataset["test"]]:
  for example in databatch:
    i= 0
    for label, sentence in example.to_labeled_lines():
      if i == 0:
        totaldata.append([sentence, label])
        break

In [9]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining
 
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [10]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 1.0MB 25.1MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=605042fa2e3cc6295920f5e8d6ba87d63e7e4cfefa483ef04506911964006d4b
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [11]:
import random
import googletrans

def backtranslate(sentence):
  translator = googletrans.Translator()

  available_langs = list(googletrans.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

  translations = translator.translate(sentence, dest=trans_lang).text 
  #print(translations)

  translations_en_random = translator.translate(translations, src=trans_lang, dest='en').text
  #en_text = [t.text for t in translations_en_random]
  return translations_en_random
  print(en_text)

In [12]:
import collections
import matplotlib.pyplot as plt

def print_frequency(data):
  x, y = list(zip(*data))
  # frequency of labels
  label_frequency = collections.Counter(y)
  print("frequency distribution: ", label_frequency)

print_frequency(totaldata)

frequency distribution:  Counter({1: 3140, 3: 3111, 2: 2242, 4: 1852, 0: 1510})


In [13]:
split_percentage = 0.75

In [14]:
def augment_data(sentence):
  n = random.randint(1,3)
  if n == 1:
    sentence = ''.join(random_deletion(list(sentence)))
  elif n == 2:
    sentence = ''.join(random_swap(list(sentence)))
  else:
    sentence = backtranslate(sentence)
  return sentence


In [96]:
def get_label_elements(trainlist, label):
   return [x for x in trainlist if x[1] == label]

def get_int_index(split, list_to_sample):
  return int(split*len(list_to_sample))

In [305]:
import random
random.shuffle(totaldata)

split_index = get_int_index(split_percentage,totaldata )

trainlist = totaldata[:split_index]
testlist = totaldata[split_index:]

iterator = 0
 
changedsent = 0
length = len(trainlist)
iterator = 0

print_frequency(trainlist)

frequency distribution:  Counter({1: 2360, 3: 2317, 2: 1672, 4: 1392, 0: 1150})


In [306]:
train_list_aug = random.sample(trainlist, int(augmentation_percentage*len(trainlist)))

train_very_negative = get_label_elements(trainlist, 0)
train_negative = get_label_elements(trainlist, 1)
train_neutral = get_label_elements(trainlist, 2)
train_positive = get_label_elements(trainlist, 3)
train_very_positive = get_label_elements(trainlist, 4)

train_list_aug = [x for label_list in [random.sample(train_very_negative, get_int_index(1, train_very_negative)), 
                              random.sample(train_negative, get_int_index(0.2, train_negative)), random.sample(train_neutral, get_int_index(0.2, train_neutral)), 
                              random.sample(train_positive, get_int_index(0.5, train_positive)), random.sample(train_very_positive, get_int_index(0.8, train_very_positive))] for x in label_list ]
len(train_list_aug)

for i in train_list_aug:
    if iterator%100 == 0:
        print("sentences changed: ", iterator)
    iterator +=1
    x,y = i
    x = augment_data(x)
    trainlist.append((x, y))

print_frequency(trainlist)


sentences changed:  0
sentences changed:  100
sentences changed:  200
sentences changed:  300
sentences changed:  400
sentences changed:  500
sentences changed:  600
sentences changed:  700
sentences changed:  800
sentences changed:  900
sentences changed:  1000
sentences changed:  1100
sentences changed:  1200
sentences changed:  1300
sentences changed:  1400
sentences changed:  1500
sentences changed:  1600
sentences changed:  1700
sentences changed:  1800
sentences changed:  1900
sentences changed:  2000
sentences changed:  2100
sentences changed:  2200
sentences changed:  2300
sentences changed:  2400
sentences changed:  2500
sentences changed:  2600
sentences changed:  2700
sentences changed:  2800
sentences changed:  2900
sentences changed:  3000
sentences changed:  3100
sentences changed:  3200
sentences changed:  3300
sentences changed:  3400
sentences changed:  3500
sentences changed:  3600
sentences changed:  3700
sentences changed:  3800
sentences changed:  3900
sentences ch

In [307]:
example1 = [data.Example.fromlist([trainlist[i][0], trainlist[i][1]], fields) for i in range(len(trainlist))]

In [308]:
example2 = [data.Example.fromlist([testlist[i][0], testlist[i][1]], fields) for i in range(len(testlist))]

In [289]:
SEED = 230
torch.manual_seed(SEED)

In [290]:
train = data.Dataset(example1, fields)
valid = data.Dataset(example2, fields)

In [291]:
(len(train), len(valid))

(14009, 2964)

In [292]:
Review.build_vocab(train)
Label.build_vocab(train)

In [293]:
print('Size of input vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

print('label freq:', list(Label.vocab.freqs.most_common(6)))

Size of input vocab :  34974
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 12238), (',', 11137), ('the', 8499), ('a', 6559), ('of', 6303), ('and', 6255), ('to', 4353), ('-', 4272), ("'s", 3669), ('is', 3604)]
Labels :  defaultdict(None, {1: 0, 3: 1, 2: 2, 4: 3, 0: 4})
label freq: [(1, 3055), (3, 3023), (2, 2827), (4, 2776), (0, 2328)]


In [294]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [295]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 64, 
                                                            sort_key = lambda x: len(x.reviews),
                                                            sort_within_batch=True, device = device)

In [80]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [81]:
!cd /content/drive/MyDrive/assign5

In [84]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Review.vocab.stoi, tokens)

In [89]:
with open('train.pkl', 'wb') as tokens: 
    pickle.dump(train.examples, tokens)

In [86]:
with open('valid.pkl', 'wb') as tokens: 
    pickle.dump(valid.examples, tokens)

In [309]:
import torch.nn as nn
import torch.nn.functional as F
 
class classifierLSTM(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional = False
                           )
        
        self.dropout = nn.Dropout(dropout)
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        embedded = self.dropout(embedded)
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   

        dense_outputs = self.dropout(dense_outputs)
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output#dense_outputs[0]

In [248]:
import torch.nn as nn
import torch.nn.functional as F
 
class classifierLSTM_v2(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional = False
                           )
        
        self.dropout = nn.Dropout(dropout)
        
        self.fc1 = nn.Linear(hidden_dim, output_dim*5)

        self.fc2 = nn.Linear(output_dim*5, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        embedded = self.dropout(embedded)
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs1 = self.fc1(hidden)   

        dense_outputs1 = self.dropout(dense_outputs1)

        dense_outputs = self.fc2(dense_outputs1)
        dense_outputs = self.dropout(dense_outputs)
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output#dense_outputs[0]

In [339]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.35
lr = 1e-4
# Instantiate the model
model = classifierLSTM(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [340]:
print(model)
 
#No. of trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifierLSTM(
  (embedding): Embedding(34974, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.35)
  (dropout): Dropout(p=0.35, inplace=False)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 10,734,305 trainable parameters


In [341]:
import torch.optim as optim
 
# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [342]:
def get_pred_vs_act_per_label(pred_act):
  preds, act = list(zip(*pred_act))
  predictions = []
  actuals = []
  for i in preds:
    _, pred = torch.max(i, 1)
    for j in pred:
      predictions.append(j.item())
  for i in act:
    for j in i:
      actuals.append(j.item())

  assert len(predictions) == len(actuals)

  label_dict = {}
  
  for i in range(0, len(actuals)):
    predic = predictions[i]
    actual = actuals[i]
    if label_dict.get(actual, None) != None:
      in_dict = label_dict[actual]
      if in_dict.get(predic, None) != None:
        in_dict[predic] += 1
      else:
        in_dict[predic] = 1
    else:
      in_dict = {predic: 1}
    label_dict[actual] = in_dict
  print("label distribution: actual vs pred:", label_dict)
  return label_dict

In [343]:
import torch.nn.functional as Fun

def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        review, review_lengths = batch.reviews   
        
        # convert to 1D tensor
        beforesqueeze = model(review, review_lengths)  
        predictions = beforesqueeze.squeeze()
        # compute the loss       
        
        loss = criterion(predictions, batch.labels)        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [344]:
def evaluate(model, iterator, criterion, preds_actual_tup):
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0
 
    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            review, review_lengths = batch.reviews
            #print(review)
            # convert to 1d tensor
            predictions = model(review, review_lengths).squeeze()
            #print(predictions)
            # compute loss and accuracy
            preds_actual_tup.append((predictions, batch.labels))
            loss = criterion(predictions, batch.labels)

            #print(batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [345]:
N_EPOCHS = 300
best_valid_loss = float('inf')


epochi = 0
train_acc_list = []
valid_acc_list = []

for epoch in range(N_EPOCHS):

    epochi += 1 
    
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    train_acc_list.append(train_acc)

    preds_actual_tup = []
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, preds_actual_tup)
    valid_acc_list.append(valid_acc)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    print(epochi)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')
    if epochi%20 == 0:
      print(f'\t pred vs actuals: {get_pred_vs_act_per_label(preds_actual_tup)}')

1
	Train Loss: 1.607 | Train Acc: 22.11%
	 Val. Loss: 1.593 |  Val. Acc: 26.97% 

2
	Train Loss: 1.605 | Train Acc: 22.50%
	 Val. Loss: 1.589 |  Val. Acc: 27.76% 

3
	Train Loss: 1.604 | Train Acc: 23.13%
	 Val. Loss: 1.588 |  Val. Acc: 28.59% 

4
	Train Loss: 1.603 | Train Acc: 23.62%
	 Val. Loss: 1.585 |  Val. Acc: 29.61% 

5
	Train Loss: 1.601 | Train Acc: 24.35%
	 Val. Loss: 1.584 |  Val. Acc: 29.16% 

6
	Train Loss: 1.598 | Train Acc: 25.25%
	 Val. Loss: 1.581 |  Val. Acc: 29.57% 

7
	Train Loss: 1.596 | Train Acc: 25.77%
	 Val. Loss: 1.579 |  Val. Acc: 29.93% 

8
	Train Loss: 1.592 | Train Acc: 26.73%
	 Val. Loss: 1.576 |  Val. Acc: 30.80% 

9
	Train Loss: 1.586 | Train Acc: 27.76%
	 Val. Loss: 1.570 |  Val. Acc: 30.23% 

10
	Train Loss: 1.579 | Train Acc: 28.54%
	 Val. Loss: 1.570 |  Val. Acc: 30.21% 

11
	Train Loss: 1.571 | Train Acc: 30.07%
	 Val. Loss: 1.567 |  Val. Acc: 30.28% 

12
	Train Loss: 1.566 | Train Acc: 30.35%
	 Val. Loss: 1.564 |  Val. Acc: 30.15% 

13
	Train Los

In [337]:
print(train_acc_list)
print(valid_acc_list)

[0.21265671310359485, 0.22230608632031096, 0.2361561724040062, 0.23688716648920485, 0.2396972382177501, 0.24672429501738177, 0.2605668709158353, 0.2628937855567018, 0.2733630237241858, 0.27854507127308953, 0.2926630217190747, 0.30398091401683686, 0.31339747053847467, 0.3303605403802166, 0.3279973263065565, 0.3382975846937258, 0.35068568247094, 0.34896459182103473, 0.35952395255162833, 0.37349795723614626, 0.3742389650377509, 0.38026716338989397, 0.3799367138784226, 0.39139730037619536, 0.3939294941621284, 0.4010028639068342, 0.4075755527302555, 0.4113556938628628, 0.4121242390103536, 0.42148697240167554, 0.42432332977856674, 0.4318585776847247, 0.4413665083989705, 0.43470494877802185, 0.4433642252939477, 0.45637567103181254, 0.4533790956893468, 0.454172674923727, 0.46047875108239855, 0.4656507851870637, 0.468791306154913, 0.472080779946558, 0.47868976813473113, 0.4849219940024424, 0.48837544058011545, 0.4958919128870855, 0.4939930806421254, 0.4953211366313778, 0.5026260715097053, 0.502

In [50]:
def get_pred_vs_act_per_label(pred_act):
  preds, act = list(zip(*pred_act))
  predictions = []
  actuals = []
  for i in preds:
    _, pred = torch.max(i, 1)
    for j in pred:
      predictions.append(j.item())
  for i in act:
    for j in i:
      actuals.append(j.item())

  assert len(predictions) == len(actuals)

  label_dict = {}
  
  for i in range(0, len(actuals)):
    predic = predictions[i]
    actual = actuals[i]
    if label_dict.get(actual, None) != None:
      in_dict = label_dict[actual]
      if in_dict.get(predic, None) != None:
        in_dict[predic] += 1
      else:
        in_dict[predic] = 1
    else:
      in_dict = {predic: 1}
    label_dict[actual] = in_dict
  print("label distribution: actual vs pred:", label_dict)
  return label_dict

In [232]:
preds, act = list(zip(*preds_actual_tup))
zero_preds = 0
same_probability = 0
for i in preds:
  for j in i:
    maxv, pred = torch.max(j, dim = 0)
    minv, pred = torch.min(j, dim = 0)
    if pred == 0:
      zero_preds += 1
      if maxv - minv < 0.1:
        same_probability += 1
      
zero_preds, same_probability

(123, 0)

In [352]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')



def classify_review(review):
    
    categories = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3:"Positive", 4:"Very Positive"}
    
    # tokenize the review 
    tokenized = [tok.text for tok in nlp.tokenizer(review)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()], pred.item()

In [365]:
def classify_and_print(test_list):
  for i in test_list:
    x,y = i
    cat  = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3:"Positive", 4:"Very Positive"}
    actual_label = cat[y]
    predicted_label_str, predicted_label = classify_review(x) 
    print("sentence: ", x)
    print("actual_label: ", actual_label, "predicted_label: ", predicted_label_str)
    print("")

In [366]:
testdata_to_test = random.sample(testlist, 25)
classify_and_print(testdata_to_test)

sentence:  Possibly not since Grumpy Old Men have I heard a film so solidly connect with one demographic while striking out with another .
actual_label:  Neutral predicted_label:  Very Negative

sentence:  Burns never really harnesses to full effect the energetic cast .
actual_label:  Negative predicted_label:  Negative

sentence:  ( Screenwriter ) Pimental took the Farrelly Brothers comedy and feminized it , but it is a rather poor imitation .
actual_label:  Neutral predicted_label:  Negative

sentence:  Unless you 're a fanatic , the best advice is : ` Scooby ' do n't .
actual_label:  Negative predicted_label:  Very Positive

sentence:  The film takes the materials of human tragedy and dresses them in lovely costumes , Southern California locations and star power .
actual_label:  Neutral predicted_label:  Negative

sentence:  It is philosophy , illustrated through everyday events .
actual_label:  Positive predicted_label:  Very Negative

sentence:  Daughter From Danang reveals that e

In [368]:
count = 0
false_pos_list = []
for i in testlist:
  x,y = i
  _, predicted_label = classify_review(x) 
  if predicted_label in [3,4] and y in [0,1]:
    false_pos_list.append(i)
    count += 1
    if count == 10:
      break

print(false_pos_list)

[["The movie is silly beyond comprehension , and even if it were n't silly , it would still be beyond comprehension .", 0], ['One of those decades-spanning historical epics that strives to be intimate and socially encompassing but fails to do justice to either effort in three hours of screen time .', 1], ['Alas , getting there is not even half the interest .', 0], ["It 's quite diverting nonsense .", 1], ["... Liotta is put in an impossible spot because his character 's deceptions ultimately undo him and the believability of the entire scenario .", 1], ['After the first 10 minutes , which is worth seeing , the movie sinks into an abyss of clichés , depression and bad alternative music .', 1], ['Witless and utterly pointless .', 1], ["The filmmakers juggle and juxtapose three story lines but fail to come up with one cogent point , unless it 's that life stinks , especially for sensitive married women who really love other women .", 1], ['I hated every minute of it .', 0], ['Why he was g

In [369]:
classify_and_print(false_pos_list)

sentence:  The movie is silly beyond comprehension , and even if it were n't silly , it would still be beyond comprehension .
actual_label:  Very Negative predicted_label:  Positive

sentence:  One of those decades-spanning historical epics that strives to be intimate and socially encompassing but fails to do justice to either effort in three hours of screen time .
actual_label:  Negative predicted_label:  Positive

sentence:  Alas , getting there is not even half the interest .
actual_label:  Very Negative predicted_label:  Positive

sentence:  It 's quite diverting nonsense .
actual_label:  Negative predicted_label:  Positive

sentence:  ... Liotta is put in an impossible spot because his character 's deceptions ultimately undo him and the believability of the entire scenario .
actual_label:  Negative predicted_label:  Positive

sentence:  After the first 10 minutes , which is worth seeing , the movie sinks into an abyss of clichés , depression and bad alternative music .
actual_labe

In [ ]:

for i in testdata_to_test:
  x,y = i
  cat  = {0: "Very Negative", 1: "Negative", 2: "Neutral", 3:"Positive", 4:"Very Positive"}
  actual_label = cat[y]
  predicted_label = classify_review(x) 
  print("sentence: ", x)
  print("actual_label: ", actual_label, "predicted_label: ", predicted_label)
  print("")

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [50]:
torch.tensor([1.0000e+00, 9.0564e-22, 7.4852e-13, 5.6629e-07, 4.7160e-15],
       device='cuda:0').shape

torch.Size([5])

In [124]:
preds2 = []
for i in preds:
  _, predictions = torch.max(i, 1)
  #print(predictions)
  #break
  for j in predictions:
    preds2.append(j)

In [207]:
len(act)

186

In [273]:
dic = get_pred_vs_act_per_label(preds_actual_tup)

label distribution: actual vs pred: {3: {1: 194, 4: 32, 3: 105, 0: 194, 2: 33}, 2: {2: 202, 1: 203, 3: 23, 0: 32, 4: 6}, 1: {3: 84, 1: 426, 0: 127, 4: 26, 2: 143}, 0: {0: 354, 3: 121, 4: 80, 1: 159, 2: 27}, 4: {1: 72, 3: 41, 0: 201, 4: 59, 2: 20}}


In [274]:
final_su = 0
final_co = 0
for keys in dic:
  key = dic[keys]
  su = 0
  predicted = 0
  for keys2 in key:
     su += key[keys2]
     if keys2 == keys:
       predicted = key[keys2]
  print(keys, su, predicted)
  final_su += su
  final_co += predicted
print(final_su, final_co)
print(final_co/final_su)

3 558 105
2 466 202
1 806 426
0 741 354
4 393 59
2964 1146
0.3866396761133603


In [260]:
import pandas as pd
train_df = pd.DataFrame(trainlist, columns = ['sentence', 'label'])
test_df = pd.DataFrame(testlist, columns = ['sentence', 'label'])
train_df['label'].value_counts(), test_df['label'].value_counts()

(1    2365
 3    2332
 2    1696
 4    1350
 0    1148
 Name: label, dtype: int64, 3    779
 1    775
 2    546
 4    502
 0    362
 Name: label, dtype: int64)

In [240]:
trainlist[0:10]

[]

In [ ]:
t = torch.tensor([0, 3, 1, 0, 3, 1, 0, 3, 2, 0, 1, 1, 3, 1, 4, 0, 4, 4, 0, 2, 2, 1, 3, 4,
        0, 1, 0, 4, 3, 0, 1, 4])


In [ ]:
F.softmax(torch.tensor([-8.7747e-02, -4.0806e-02, -8.3384e-02,  3.9979e-02, -6.0466e-02]), dim=1)

IndexError: ignored

In [ ]:
import torch.nn.functional as f
#t.apply_(lambda x : torch.nn.functional.one_hot(torch.tensor(x))) #torch.nn.functional.one_hot(torch.tensor(x), 5))

t = t.tolist()
print(t)
oneh = []
for i in t:
  print(i)
  oneh.append(f.one_hot(torch.tensor(i), 5).tolist())



In [ ]:
torch.tensor(oneh)

tensor([[1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1]])

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

input, target

(tensor([[ 0.5062,  0.1430, -0.0579,  0.2270, -0.0728],
         [ 1.7862,  0.0257, -1.2147,  0.4436,  1.6470],
         [ 0.3827, -1.1616, -1.6744,  1.2986, -1.1603]], requires_grad=True),
 tensor([4, 1, 3]))

In [ ]:
t = torch.tensor([[0.3525, 0.3786, 0.1276, 0.0825, 0.0589],[0.3653, 0.3919, 0.1250, 0.0693, 0.0485],[0.3750, 0.3844, 0.1240, 0.0717, 0.0448]])
x, y = torch.max(t, 1)
x,y

(tensor([0.3786, 0.3919, 0.3844]), tensor([1, 1, 1]))

In [ ]:
t= torch.tensor([1])
t

tensor([1])

In [ ]:
import torch
torch.nn.functional.one_hot(torch.tensor(2), 6)

tensor([0, 0, 1, 0, 0, 0])

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.